# RV calculation at specific times.

To determine the RV of the planets expected in my atmospheres

In [ ]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt

import sys
#sys.path.append('/home/jneal/azores/planet/rvs/exonailer/utilities')

%pylab inline

In [ ]:
#Rodrigo Diaz's true anomaly calculation

import numpy as np

def trueanomaly(ma, ecc, niterationmax=10000):
    """
    Compute the true anomaly using the Newton-Raphson method.

    :param array-like ma: mean anomaly.
    :param float ecc: orbital eccentricity.
    :param int niterationmax: maximum number of iterations for N-R method.
    """
    
    if not isinstance(ma, float):
        ea = ma
    else:
        ea = np.array([ma,])

    # Initialise at ea0 = ma
    niteration = 0
    ea0 = ma
    
    while np.linalg.norm(ea - ea0, ord=1) > 1e-5 or niteration==0:
        ea0 = ea
 
        ff = ea - ecc*np.sin(ea) - ma   # Function
        dff = 1 - ecc*np.cos(ea)        # Derivative

        # Use Newton method
        ea = ea0 - ff / dff

        # Increase iteration number; if above limit, break with exception.
        niteration += 1
        if niteration >= niterationmax:
            raise RuntimeError('Eccentric anomaly computation'
                               'not converged.')
        
    # Compute true anomaly from eccentric anomaly
    return 2. * np.arctan2(np.sqrt(1. + ecc) * np.sin(ea/2.),
                           np.sqrt(1. - ecc) * np.cos(ea/2.))

# Maen Anomaly calculation
def meananomaly(t, T0, P):
    """ Calculate mean Anomaly using period, tau and a time value"""
    return 2 * np.pi * (t - T0) / P

def radialvelocity(gamma, K, ta, omega, ecc):
    # Calculate radial velocity of star
    return gamma + K *(np.cos(ta + omega) + ecc * np.cos(omega))

def rv_curve_py(t, gamma, K, omega, ecc, T, P):
    ma = meananomaly(t, T, P)
    ta = trueanomaly(ma, ecc)
    rv = radialvelocity(gamma, K, ta, omega, ecc)
    return rv
    
    

In [ ]:
#turn 1997.04 +- 0.02 into julian days
print(0.04 * 365)
print(.6*24)
print(.4*60)
# Julian date for CE  1997 January 14 14:24:00.0 UT is
# JD 2450463.100000

# Keplerian Orbital Parameters:

In [ ]:


#Target parameters:  Obtained from sahlmann et al.
#                 [mean_val, K1, omega, e, Tau, Period]

#HD4747_params   = [9.904, 703.3, -94.2, 0.723, 62059.1, 11593.2] # Best fit solution from sahlmann not complete period coverage
#Crepp 2016 updated HD4747 parameters  t = 1997.04

HD4747_params   = [-0.219, 755.3, -69.1, 0.740, 50463.10, 37.88*365.25] 
HD162020_params = [-27.328, 1813, 28.40, 0.277, 51990.677, 8.428198]  # here mean val in km/s K in m/s https://arxiv.org/pdf/astro-ph/0202458v2.pdf
HD167665_params = [8.003, 609.5, -134.3 ,0.340, 56987.6, 4451.8]  # sahlman
HD211847_params = [6.689, 291.4, 159.2, 0.685, 62030.1, 7929.4]  # Best fit solution from sahlmann not complete period coverage
HD30501_params =  [23.710, 1703.1, 70.4, 0.741, 53851.5, 2073.6]  # sahlman


# 2 companions
# One of these used 3 body equations - need to compare relative strengths
HD168443b_params = [-0.046533, 475.133, 172.923, 0.52883, 15626.199, 58.1125] #  % msini = 7.659Mjp, a=0.2931AU, dv/dt (ms−1 yr−1) −0.00868
HD168443c_params = [-0.046533, 297.70, 64.87, 0.2113, 15521.3, 1749.83] 
#singal companion- old model
#HD202206_params = [14.681, 564.8, 160.32, 0.429, 51919.02, 256.003] #drift of 42.9 ms per year!    # here mean val in km/s K in m/s
HD202206b_params = [14.721, 564.75, 161.18, 0.435, 52250.00, 255.87] #\citet{Correia2005_hd202206bc}\\  % \lambda = 266.228 deg (mean longitude)
HD202206c_params = [14.721, 42.01, 78.9, 0.267, 52250.00, 1383.4] # \citet{Correia2005_hd202206bc}\\ % \lambda = 30.586 deg (mean longitude)


#Median times 
# extremeties are +- 14 minutes
HD30501_times =  [2456024.505902, 2456140.88716, 2456141.86633, 2456145.904258]  # from data - need to double check with calculated data centers
HD162020_times = [2456112.76624, 2456112.79015]
HD202206_times = [ 2456120.78801, 2456121.73727, 2456119.85411]   
HD211847_times = [0, 2456121.78793]   # Need to find out the first value here. 
HD4747_times =   [2456114.81674]
HD167665_times = [2456136.70895, 2456136.73434, 2456144.62087] 
HD168443_times = [2456144.68718, 2456144.70753]
params_dict = {"HD30501":HD30501_params, "HD211847":HD211847_params, \
               "HD4747":HD4747_params, "HD167665":HD167665_params, \
               "HD162020":HD162020_params, \
               "HD202206b":HD202206b_params,"HD202206c":HD202206c_params, \
               "HD168443b":HD168443b_params,"HD168443c":HD168443c_params}
               # "HD202206":HD202206_params,
    
times_dict = {"HD30501":HD30501_times, "HD162020":HD162020_times, \
              "HD202206":HD202206_times, "HD211847":HD211847_times, \
              "HD4747":HD4747_times, "HD167665":HD167665_times, \
             "HD168443":HD168443_times}


In [ ]:
def RV_from_params(t, params):
    """ Get RV values with parameter list.
    
    input:
        t -- The time/s at which to calculate the RV value
        params -- a list of values [mean_val, K1, omega, e, Tau, Period]
    
    omega should be given in degrees. This function converts it to radians.
    
    Outputs:
        RVs -- The radial velocity values evaluated at the given times.
    
    """
    if not isinstance(t, np.ndarray):
        t = np.array(t)
    else:
        pass
    params = params[:]
    params[0] = params[0] * 1000 # Turn into m/s
    params[2] = np.deg2rad(params[2])
    #print(params)
    # Note: test that is np.deg2rad is faster than *np.pi /180
    try: # Try rv_curve first
        RVs = rv_curve(t, *params) # *unpacks parameters from list
    except:
        RVs = rv_curve_py(t, *params) # *unpacks parameters from list
    return RVs


def plot_RV_phase_curve(params, name=False, t_vals=False):
    """Plot RV phase curve use period and T0 to get times from phase.
    
    params -- a list of values [mean_val, K1, omega, e, Tau, Period]"""
    
    phase = np.linspace(-0.1,1.1, 200)
    t = params[4] + phase*params[5]
    
    rv_values = RV_from_params(t, params)
    
    plt.figure()
    plt.plot(phase, rv_values)
    if name:
        plt.title("RV Phase curve for {}".format(name))
    else:
        plt.title("RV Phase curve")
    if t_vals:
        for t_num, t_val in enumerate(t_vals):
            phi = (t_val - params[4])/params[5]  % 1
            rv = RV_from_params(t_val, params)
            plt.plot(phi, rv, "+",label = "Obs #{0}".format(t_num), markersize=16, markeredgewidth=3)
            plt.legend(loc=0)
    plt.xlabel("Phase")
    plt.ylabel("RV amplitude (m/s)")
    plt.show()


In [ ]:
RV_HD30501 = RV_from_params(HD30501_times, HD30501_params)
print(HD30501_times)
print(RV_HD30501)


In [ ]:
RV_HD4747 = RV_from_params(HD4747_times, HD4747_params)
print(np.array(HD4747_times) - 2440000)
print(RV_HD4747)
plot_RV_phase_curve(HD4747_params, t_vals=HD4747_times)

In [ ]:
plot_RV_phase_curve(HD202206b_params, t_vals=HD202206_times)
plot_RV_phase_curve(HD202206c_params, t_vals=HD202206_times)

In [ ]:
plot_RV_phase_curve(HD211847_params, t_vals=HD211847_times)

In [ ]:
plot_RV_phase_curve(HD162020_params, t_vals=HD162020_times)

In [ ]:
obs_time_sigma = 14/(60*24)  # +-14min

In [ ]:
# Test RV determination 

# Get parameters for star and taget
Red_JD = 2400000
#hd30501
mean_val = 23.710 # +- 0.017
Period = 2073.6   # +3 - 2.9 days
e = 0.741         # +_ 0.04
K1 = 1703.1       #+- 26  ms^-1
Tau = 53851.5      # +- 3 JD    Note: I think this is tau but I am not sure. 
omega = 70.4 * np.pi/180     # +- 0.7 deg

for ti in np.array([2456024.505902, 2456140.887153, 2456141.866329, 2456145.904258]) - Red_JD:
   # Calculate Mean anomaly:
   M_anomaly = meananomaly(ti, Tau, Period)
   #print("Mean Anomaly = {}".format(M_anomaly))

   # Determine true anomaly from mean anomaly
   true_anomaly = trueanomaly(M_anomaly, e)
   #print("Daiz true anomaly = {0}".format(true_anomaly))
   #nu goes into RV equations

   # Now Calculalte the RV
   RV = mean_val + K1 *(np.cos(true_anomaly + omega) + e * np.cos(omega))

   print("RV value {0} ms^-1 at time {1} JD*".format( RV ,  ti ))

In [ ]:

times = np.linspace(Tau, Tau+Period, 1000)

M_anomalies = Mean_Anomaly(times, Tau, Period)



In [ ]:

plt.plot(M_tests-M_anomalies)
plt.title("Differences between M_tests and M_anomalies")
plt.show()
#print(lmfit.report_fit(out))

T_anomalies = True_Anomaly(E_anomalies, e)
T_anomalies2 = True_Anomaly2(E_anomalies, e)
T_anomalies3 = True_Anomaly3(E_anomalies, e)
T_anomalies4 = True_Anomaly4(E_anomalies, e)

rod_true = trueanomaly(M_anomalies, e)
print("done")

In [ ]:
#print(T_anomalies)
#print(T_anomalies2)
#print(T_anomalies3)
#plt.plot(times, T_anomalies, label="1")
#plt.plot(times, T_anomalies2, label="2")
#plt.show()

In [ ]:
RVs_1 = Radial_velocity(mean_val, K1, T_anomalies, omega, e)
RVs_2 = Radial_velocity(mean_val, K1, T_anomalies2, omega, e)
RVs_3 = Radial_velocity(mean_val, K1, T_anomalies3, omega, e)
RVs_4 = Radial_velocity(mean_val, K1, T_anomalies4, omega, e)
RVs_true = Radial_velocity(mean_val, K1, rod_true, omega, e)

In [ ]:
# Test ajplanet comparision

#mean_val, K1, T_anomalies4, omega, e
#ajplanet_rvs = rv_curve(t, gamma,K,omega,ecc,to,P)
ajplanet_rvs = rv_curve(times, mean_val, K1, omega, e, Tau, Period)

print(rv_curve(HD30501_times, mean_val, K1, omega, e, Tau, Period))

In [ ]:
plt.plot((times-Tau) / Period, RVs_1, label="1")
plt.plot((times-Tau) / Period, RVs_2,'+', label="2")
plt.plot((times-Tau) / Period, RVs_3, 'o', label="3")
plt.plot((times-Tau) / Period, RVs_4, "--", label="4")
plt.plot((times-Tau) / Period, RVs_true, "--", label="4")
#plt.plot((times-Tau) / Period, ajplanet_rvs, "k*", label="ajplanet")
plt.title("Radial Velocities")
plt.legend(loc=0)
plt.show()

In [ ]:
#plt.plot(T_anomalies, RVs_1)
#plt.plot(T_anomalies2, RVs_2)
#plt.show()

In [ ]:

# My Values are not good atm when compaing to rvcurve. 

In [ ]:
# Try thre true anomaly given by the lecuture on facebook.

In [ ]:
## Old code to caluclate true anomaly from mean anomaly



